<a href="https://colab.research.google.com/github/MengOonLee/WebScrapy/blob/master/Groceries/Lotus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
apt install chromium-chromedriver

pip install --no-cache-dir -qU scrapy selenium

In [39]:
%%writefile Lotus.py
from selenium import webdriver
import scrapy
from scrapy.crawler import CrawlerProcess

class LotusSpider(scrapy.Spider):
    name = 'Lotus'

    def start_requests(self):
        urls = [
            'https://v17.angular.io/docs'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse,
                meta={'url': url})

    def parse(self, response):
        url = response.meta['url']
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(options=options)
        driver.get(url)

        title = driver.title
        print(title)

process = CrawlerProcess()
process.crawl(LotusSpider)
process.start()

Overwriting Lotus.py


In [ ]:
%%bash
python Lotus.py